In [1]:
import sys 

from vmtk import pypes
from vmtk import vmtkscripts

In [2]:
mast_dir = ' D:/Hartmann/PythonAll/seal_whiskers/Har20191023_harbor3_jig1/' 
dir_1 = mast_dir + 'saved/IMG-0002-00001.dcm ' ###raw dcm stack
dir_2 = mast_dir + 'image_volume_voi.vti ' ###image: one whisker
dir_5 = mast_dir + 'image_volume_voi_v2.vti ' ###image: many whisker
dir_3 = mast_dir + 'mc_surface.vtp ' ### surface
dir_4 = mast_dir + 'level_sets.vti ' ###image 

arg_1 = 'vmtkimagereader -ifile' + dir_1 + '--pipe vmtkimagevoiselector -ofile' +  dir_5
arg_3 = 'vmtklevelsetsegmentation -ifile' + dir_5 + '-ofile' + dir_4
arg_2 = 'vmtkmarchingcubes -ifile' +  dir_5 +'-l 5.0 -ofile' +  dir_3 + '--pipe vmtksurfaceviewer'
myArguments = 'vmtkmarchingcubes -ifile ' + dir_2 + ' -l 5 --pipe vmtksurfaceviewer'

#myPype = pypes.PypeRun(myArguments)

In [3]:
myPype = pypes.PypeRun(arg_1)


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = D:/Hartmann/PythonAll/seal_whiskers/Har20191023_harbor3_jig1/saved/IMG-0002-00001.dcm
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = D:/Hartmann/PythonAll/seal_whiskers/Har20191023_harbor3_jig1/saved/IMG-0002-00001.dcm
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 0.034357 0.034344 0.034370
Origin -28.459585 -33.282825 -126.882812
Dimensions 1758 1814 931
Done executing vmtkimagereader.
Output vmtkimagereader members:


In [ ]:
mySurface = myPype.GetScriptObject('vmtkmarchingcubes','0').Surface
mySmoother = vmtkscripts.vmtkSurfaceSmoothing()
mySmoother.Surface = mySurface
mySmoother.PassBand = 0.1
mySmoother.NumberOfIterations = 30
mySmoother.Execute()

In [ ]:
myWriter = vmtkscripts.vmtkSurfaceWriter()
myWriter.Surface = mySmoother.Surface
myWriter.OutputFileName = 'mysurface.vtp'
myWriter.Execute()

In [ ]:
customscript = 'customScript'

class customScript(pypes.pypeScript):
    def __init__(self):
        pypes.pypeScript.__init__(self)
        self.Image = None
        self.Surface = None
        self.Levels=[]
        self.SetScriptName('customScript')
        self.SetScriptDoc('here goes a description')
        self.SetInputMembers([
            ['Image','i','vtkImageData',1,'','the input image','vmtkimagereader'],
            ['Levels','levels','float',-1,'','graylevels to generate the isosurface at'],
            ])
        self.SetOutputMembers([
            ['Surface','o','vtkPolyData',1,'','the output surface','vmtksurfacewriter']
            ])
    def Execute(self): 
        if self.Image == None:
            self.PrintError('Error: No Image.')
        if self.Levels == []:
            self.PrintError('Error: No Levels')
        self.marchingCubes = vmtkscripts.vmtkMarchingCubes()
        self.marchingCubes.Image = self.Image
        self.marchingCubes.Connectivity = 1
        self.vmtkRenderer = vmtkscripts.vmtkRenderer()
        self.vmtkRenderer.Initialize()
        self.SurfaceViewer = vmtkscripts.vmtkSurfaceViewer()
        self.SurfaceViewer.vmtkRenderer = self.vmtkRenderer
        for level in self.Levels:
            self.marchingCubes.Level = level
            self.marchingCubes.Execute()
            self.Surface = self.marchingCubes.Surface
            self.SurfaceViewer.Surface = self.Surface
            self.SurfaceViewer.BuildView()

if __name__=='__main__':
main = pypes.pypeMain()
main.Arguments = sys.argv
main.Execute()